In [1]:
from statistics import mode

import cv2
from keras.models import load_model
import numpy as np

Using TensorFlow backend.


In [2]:
def detect_faces(detection_model, gray_image_array):
    return detection_model.detectMultiScale(gray_image_array, 1.3, 5)

def draw_bounding_box(face_coordinates, image_array, color):
    x, y, w, h = face_coordinates
    cv2.rectangle(image_array, (x, y), (x + w, y + h), color, 2)

def apply_offsets(face_coordinates, offsets):
    x, y, width, height = face_coordinates
    x_off, y_off = offsets
    return (x - x_off, x + width + x_off, y - y_off, y + height + y_off)

def draw_text(coordinates, image_array, text, color, x_offset=0, y_offset=0,
                                                font_scale=2, thickness=2):
    x, y = coordinates[:2]
    cv2.putText(image_array, text, (x + x_offset, y + y_offset),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, color, thickness, cv2.LINE_AA)
    
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

In [3]:
emotion_labels = {
    0: 'angry',
     1: 'disgust',
     2: 'fear',
     3: 'happy',
     4: 'sad',
     5: 'surprise',
     6: 'neutral'
}

# hyper-parameters for bounding boxes shape
frame_window = 10
emotion_offsets = (20, 40)

# loading models
face_detection = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
emotion_classifier = load_model('fer2013_mini_XCEPTION.119-0.65.hdf5', compile=False)

# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]

# starting lists for calculating modes
emotion_window = []

In [4]:
# starting video streaming
cv2.namedWindow('window_frame')
video_capture = cv2.VideoCapture(0)
while True:
    bgr_image = video_capture.read()[1]
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    faces = detect_faces(face_detection, gray_image)

    for face_coordinates in faces:

        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
        gray_face = gray_image[y1:y2, x1:x2]
        gray_face2 = gray_image[y1:y2, x1:x2]
        try:
            gray_face = cv2.resize(gray_face, (emotion_target_size))
            gray_face3 = cv2.resize(gray_face, (emotion_target_size))
        except:
            continue

        gray_face = preprocess_input(gray_face, True)
        gray_face = np.expand_dims(gray_face, 0)
        gray_face = np.expand_dims(gray_face, -1)
        emotion_prediction = emotion_classifier.predict(gray_face)
        emotion_probability = np.max(emotion_prediction)
        emotion_label_arg = np.argmax(emotion_prediction)
        emotion_text = emotion_labels[emotion_label_arg]
        emotion_window.append(emotion_text)

        if len(emotion_window) > frame_window:
            emotion_window.pop(0)
        try:
            emotion_mode = mode(emotion_window)
        except:
            continue

        if emotion_text == 'angry':
            color = emotion_probability * np.asarray((255, 0, 0))
        elif emotion_text == 'sad':
            color = emotion_probability * np.asarray((0, 0, 255))
        elif emotion_text == 'happy':
            color = emotion_probability * np.asarray((255, 255, 0))
        elif emotion_text == 'surprise':
            color = emotion_probability * np.asarray((0, 255, 255))
        else:
            color = emotion_probability * np.asarray((0, 255, 0))

        color = color.astype(int)
        color = color.tolist()

        draw_bounding_box(face_coordinates, rgb_image, color)
        draw_text(face_coordinates, rgb_image, emotion_mode,
                  color, 0, -45, 1, 1)

    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    cv2.imshow('window_frame', bgr_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [5]:
np.set_printoptions(threshold=np.inf)
gray_image

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [6]:
gray_face

array([[[[-0.10588235],
         [-0.12156862],
         [-0.12156862],
         [-0.23921567],
         [-0.10588235],
         [-0.10588235],
         [-0.11372548],
         [-0.11372548],
         [-0.11372548],
         [-0.13725489],
         [-0.1607843 ],
         [-0.1607843 ],
         [-0.20784312],
         [-0.23137254],
         [-0.40392154],
         [-0.78039217],
         [-0.74117649],
         [-0.77254903],
         [-0.78823531],
         [-0.88235295],
         [-0.90588236],
         [-0.87450981],
         [-0.8509804 ],
         [-0.89019608],
         [-0.86666667],
         [-0.78823531],
         [-0.87450981],
         [-0.86666667],
         [-0.85882354],
         [-0.81176472],
         [-0.81176472],
         [-0.7647059 ],
         [-0.74901962],
         [-0.67058825],
         [-0.59215689],
         [-0.53725493],
         [-0.40392154],
         [-0.3098039 ],
         [-0.03529412],
         [ 0.98431373],
         [ 0.97647059],
         [ 0.262

In [7]:
np.set_printoptions(threshold=np.inf)
gray_face.shape

(1, 48, 48, 1)

In [8]:
gray_image.shape

(720, 1280)

In [9]:
gray_face2

array([[114, 114, 115, 114, 115, 115, 115, 114, 113, 114, 114, 113, 114,
        114, 115, 115, 114, 113, 115, 113, 113, 113, 114, 114, 114, 113,
        112, 109, 103,  99,  98,  97,  98, 101, 107, 110, 112, 113, 113,
        113, 113, 112, 113, 113, 113, 113, 113, 113, 113, 113, 113, 113,
        113, 113, 113, 112, 112, 113, 112, 111, 112, 112, 112, 112, 112,
        113, 112, 113, 113, 113, 112, 110, 110, 111, 111, 110, 110, 109,
        108, 108, 108, 109, 109, 108, 109, 108, 108, 108, 107, 108, 108,
        109, 110, 112, 111, 110, 110, 110, 109, 110, 110, 108, 108, 109,
        109, 107, 108, 107, 103, 103, 103, 102, 102, 103, 105, 103, 100,
         98,  98,  96,  95,  92,  89,  82,  77,  72,  70,  66,  62,  55,
         49,  44,  35,  30,  28,  30,  33,  42,  50,  56,  59,  61,  57,
         58,  58,  56,  57,  58,  55,  49,  42,  38,  37,  32,  32,  30,
         30,  30,  30,  30,  27,  25,  25,  24,  23,  21,  19,  18,  18,
         17,  17,  14,  14,  13,  12,  12,  13,  15

In [10]:
gray_face2.shape

(444, 404)

In [11]:
gray_face3

array([[114, 112, 112,  97, 114, 114, 113, 113, 113, 110, 107, 107, 101,
         98,  76,  28,  33,  29,  27,  15,  12,  16,  19,  14,  17,  27,
         16,  17,  18,  24,  24,  30,  32,  42,  52,  59,  76,  88, 123,
        253, 252, 161, 126, 124, 121, 118, 120, 118],
       [114, 114, 113,  98, 114, 113, 113, 111, 110,  97,  94,  60,  46,
         33,  26,  22,  19,  15,  10,   9,   9,  14,  12,  15,  17,  17,
         16,  14,  16,  17,  18,  20,  28,  31,  33,  37,  47,  49,  62,
         94, 121, 125, 122, 120, 120, 118, 119, 118],
       [114, 114, 110, 110, 112, 112, 111, 108, 104,  48,  48,  29,  29,
         34,  21,  18,  16,  12,   8,  12,  16,  17,  20,  22,  24,  21,
         20,  17,  16,  16,  16,  14,  24,  25,  25,  24,  28,  34,  45,
         43,  56, 109, 114, 116, 114, 113, 113, 114],
       [112, 114, 100, 112, 113, 112, 112, 104,  73,  60,  25,  23,  19,
         19,  17,  17,  15,  14,  19,  28,  48,  55,  52,  53,  48,  33,
         35,  32,  30,  28,  24,  1

In [12]:
gray_face3.shape

(48, 48)